# MAIN CODE

In [14]:
import requests
import datetime

thunder1=[] #name
thunder2=[] #price
location = []
description = []
time = []


for page_number in range(123,125):  #125

    url = "https://api.bunjang.co.kr/api/1/find_v2.json?q=%EC%95%84%EC%9D%B4%ED%8F%B011&order=score&page={0}&request_id=2022519113331&stat_device=w&n=100&stat_category_required=1&req_ref=search&version=4".format(page_number)
    res = requests.get(url)
    res.encoding=None
    data = res.json()
    
    for d in data['list']:
        name = d['name']
        loc = d['location']
        price = int(d['price'])
        pid = d['pid']
        unix = str(datetime.datetime.fromtimestamp(d['update_time']))
        unix = unix[0:7]

        
        excepts = ["케이스","삽니다", "매입", "구매", "구입","프로","Pro","pro", "미니", "PRO", "11PRO"]
        targets = ['배터리', '판매', '팔아요']

        if (price >= 50000) & (price < 1600000) ==True :
            if ( d['ad']==False ):
                if any(x in d["name"] for x in targets) and not any(x in d['name'] for x in excepts):
                    thunder1.append(d['name'])
                    thunder2.append(price)
                    time.append(unix)
                    loc = loc.split(' ')
                    loc = loc[0]
                    # location.append(loc)
                    if len(loc) == 0:
                        location.append('NULL')
                    else:
                        location.append(loc)

                    url2 = 'https://api.bunjang.co.kr/api/1/product/{0}/detail_info.json?version=4'.format(pid)
                    res = requests.get(url2)
                    res.encoding=None
                    data = res.json()
                
                    description.append(data['item_info']['description'])
                    


                    # print(name, price, unix, loc)

        else:
            pass

# print(thunder1,thunder2, time, location )


# Desc 에서 배터리 효율 뽑아내기

In [15]:
import re
from bs4 import BeautifulSoup

li = []



for i in range(len(description)):
    text = description[i]
    regex = '배터리\s*.*\d{2,3}'
    pattern =  re.compile(regex)
    res = pattern.findall(text)
    
    regex2 = '\d{2,3}'
    pattern2 = re.compile(regex2)
    text = str(res)
    res2 = pattern2.findall(text)
    if len(res2) == 0:
        li.append('NULL')
    else:
        li.append(res2[0])

print(li)



'''
for i in range(len(li)):    
    text = li[i]
    regex2 = '\d{2,3}'
    pattern2 = re.compile(regex2)
    res2 = pattern2.findall(text)
    li1.append(res2)
'''
#print(li1)

['98', 'NULL', '81', '79', '82', '85', 'NULL', '81', '83', 'NULL', 'NULL', 'NULL', 'NULL', '83', '79']


"\nfor i in range(len(li)):    \n    text = li[i]\n    regex2 = '\\d{2,3}'\n    pattern2 = re.compile(regex2)\n    res2 = pattern2.findall(text)\n    li1.append(res2)\n"

용량 빼내기

In [16]:
size = []

for i in range(len(description)):
    text = description[i]
    regex = '64|128'
    pattern =  re.compile(regex)
    res = pattern.findall(text)
    
    if len(res) == 0:
        size.append('NULL')
    else:
        size.append(res[0])
print(size)

['128', '128', 'NULL', '64', '128', 'NULL', 'NULL', 'NULL', '64', 'NULL', '64', '128', '128', '64', '64']


# 판다스로 데이터프레임 만들기

In [17]:
import pandas as pd
df = pd.DataFrame({'name' : thunder1,
                 'price' : thunder2,
                 'time' : time,
                  'loction' : location,
                  'battery' : li,
                  'size': size
                  })
df

,name,price,time,loction,battery,size
0,아이폰11 128G 퍼플 A+급 판매합니다 !!!!!!!!! 9313,620000,2021-06,부산광역시,98,128
1,11336 무잔상 V50 블랙 128GB 22만원에 판매합니다,220000,2021-06,울산광역시,NULL,128
2,아이폰11 퍼플 256G 배터리81% 새폰급 판매중,470000,2022-05,대구광역시,81,NULL
3,아이폰11 [화이트] S급 정상해지 64GB 판매합니다 ^^,360000,2022-05,경기도,79,64
4,아이폰11 [퍼플] A급 정상해지 128GB 판매합니다 ^^,440000,2022-05,경기도,82,128
5,아이폰 11 64기가 레드 색상 판매합니다,310000,2022-05,경기도,85,NULL
6,아이폰 11 64기가 정상해지폰 판매합니다,350000,2022-05,충청북도,NULL,NULL
7,아이폰11 퍼플 256G 배터리81% SSS급 판매해요,470000,2022-05,대구광역시,81,NULL
8,아이폰11 블랙 64G 깨끗한제품 아주 저렴히 판매합니다,360000,2022-05,경기도,83,64
9,S급 애플 아이폰11 퍼플 128GB 잔상X 기능정상 가성비끝판 저렴판매,420000,2022-05,서울특별시,NULL,NULL


# 데이터 엑셀파일로 추출

In [43]:
from pandas import ExcelWriter
df.to_excel('C:/Users/admin/Desktop/big/thunder.xlsx', index=False)